In [ ]:
import polars as pl
import os

from tqdm import tqdm
import eikon as ek

ek.set_app_key("fd36c20b01ed4ad6b3248805ae5031749349e3c9")

In [ ]:
import pandas as pd

with open(r"assets\metadata\instruments.txt", "r") as f:
    all_instruments = [x.strip("\n") for x in f]

seg_length = 5000
df = pd.DataFrame()
for instruments in [all_instruments[x:x+seg_length] for x in range(0,len(all_instruments),seg_length)]:
    t_df, err = ek.get_data(
        instruments=instruments,
        fields=["TR.ExchangeCountry"],
    )
    df = pd.concat([df, t_df], axis=0)

df

In [ ]:
df.to_csv("REF_exchangecountry.csv", index=False)

In [1]:
from source.refinitiv import Eikon2

Eikon2().download_data(country="United States of America")

100%|██████████| 52/52 [03:09<00:00,  3.64s/it]


In [ ]:
with open(r"assets\metadata\fields.txt", "r") as f:
    fields = [x.strip("\n") for x in f]

fields

In [ ]:
import pandas as pd

pd.to_datetime(df["TR.REVENUE.DATE"], format="%Y-%m-%dT%H:%M:%SZ")

In [ ]:
df, err = ek.get_data(
    instruments=["AAPL.OQ", "AMZN.OQ"],
    fields=["TR.Revenue"],
    # fields=fields,
    parameters={
        "Scale": 6,
        "SDate": 0,
        "EDate": -25,
        "FRQ": "FY",
        "Curn": "Native"
    },
    field_name=True
)
df

In [ ]:
df.loc[:, df.dtypes=="string"].columns.tolist()

In [ ]:
df.dtypes

In [ ]:
pl.read_parquet("assets\Eikon\data.parquet")

In [ ]:
from source.refinitiv import EikonHelper

eikon_helper = EikonHelper()

In [ ]:
with open(r"assets\metadata\fields.txt", "r") as f:
    fields = [x.strip("\n") for x in f]

fields

In [ ]:
import time
from tqdm import tqdm

fields, instruments = eikon_helper.get_metadata()

NO_DATE_FIELDS = []
for field in tqdm(fields):
    df, err = ek.get_data(
        instruments=["AAPL.OQ", "AMZN.OQ"],
        fields=[field, field+".DATE"],
        parameters={
            "Scale": 6,
            "SDate": 0,
            "EDate": -5,
            "FRQ": "FY",
            "Curn": "Native"
        },
        field_name=True
    )
    if err:
        NO_DATE_FIELDS.append(field)
        print(field, err)

In [ ]:
NO_DATE_FIELDS

In [ ]:
eikon_helper.extract()